In [25]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar
from pypdf import PdfReader
import re

# ***Tạo hàm xử lý các chun***

In [26]:
# Xác định format của chunk được sử dụng:

# {
#   "id": "3-2-1",
#   "values": [0.123, -0.456, ...],
#   "metadata": {
#     "subject": "Lịch sử ĐCSVN",
#     "chapter": "Chương III",
#     "chapter_title": "Chương III: Đường lối kháng chiến",
#     "section": "II.",
#     "section_title": "II. Giai đoạn đầu",
#     "subsection": "1.",
#     "subsection_title": "1. Tư tưởng chỉ đạo",
#     "content": "...",
#     "position_in_document": 73,
#     "tokens": 296,
#     "type": "theory"
#   }
# }

In [27]:
def parse_chapters(text):
    """
    Phân tích 1 dòng chương/tiêu đề, hỗ trợ các định dạng:
    - 'Chương 1 Tiêu đề...'
    - 'Chương nhập môn Tiêu đề...'
    - 'I. Tiêu đề...'
    - '1. Tiêu đề...'

    Trả về tuple: (chapter_number, chapter_title)
    Nếu không khớp format, trả về (None, None)
    """
    pattern = re.compile(
        r"""^\s*
        (                                         # Nhóm 1: định danh chương
            Chương\s+(?:nhập\s+môn|\d+)           # 'Chương nhập môn' hoặc 'Chương <số>'
            |[IVXLCDM]+\.                         # Số La Mã có dấu chấm (I., II., ...)
            |\d+\.?                                # Số thường, có thể có hoặc không dấu chấm
        )
        \s*(.*)                                    # Nhóm 2: tiêu đề còn lại
        """, re.IGNORECASE | re.VERBOSE
    )
    
    line = text.strip()
    match = pattern.match(line)
    if match:
        chapter_number = match.group(1).strip().rstrip('.') or None
        chapter_title = match.group(2).strip() or None

        if (chapter_number and len(chapter_number) > 1) or (chapter_title and len(chapter_title) > 1):
            return chapter_number, chapter_title
        else:
            return None, text
    else:
        return None, text


In [28]:
chapter_number, chapter_title = parse_chapters('KẾT LUẬN')
print(chapter_number)
print(chapter_title)

None
KẾT LUẬN


In [29]:
def split_chunk_langchain(
    chapter_line,
    section_line,
    sub_section_line,
    content_line,
    max_chars=512,
    model=None
):
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=100,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    content_chunks = splitter.split_text(content_line)

    # Xử lý metadata như cũ
    chapter_number, chapter_title = parse_chapters(chapter_line) if chapter_line else (None, None)
    section_number, section_title = parse_chapters(section_line) if section_line else (None, None)
    subsection_number, subsection_title = parse_chapters(sub_section_line) if sub_section_line else (None, None)

    if 'MỤC TIÊU' in section_line:
        chunk_type = 'TARGET'
    elif 'NỘI DUNG ÔN TẬP VÀ THẢO LUẬN' in section_line:
        chunk_type = 'EXERCISES'
    else:
        chunk_type = 'THEORY'

    result = []
    for i, content_piece in enumerate(content_chunks):
        chunk = {
            "id": f"LSD-{chapter_number if chapter_number else 0}-{section_number if section_number else 0}-{subsection_number if subsection_number else 0}-{i}",
            "values": [],
            "metadata": {
                "subject": "Lịch sử Đảng Cộng Sản Việt Nam",
                "chapter": chapter_number,
                "chapter_title": chapter_title,
                "section": section_number,
                "section_title": section_title,
                "subsection": subsection_number,
                "subsection_title": subsection_title,
                "content": content_piece.strip(),
                "tokens": len(content_piece),
                "type": chunk_type
            }
        }
        result.append(chunk)

    return result

# ***Xử lý các phần chú thích ở cuối trang***

In [30]:
pdf_path = "../data/LichSuDang/lich_su_dang.pdf"

#Tạo fulltext để chứa toàn bộ các dòng chử trong file Lịch sử Đảng
sections = []

# Lấy số trang trong file PDF
reader = PdfReader(pdf_path)
total_pages = len(reader.pages)
print("Số trang trong PDF:", total_pages)

Số trang trong PDF: 219


In [31]:
pdf_path = "../data/LichSuDang/lich_su_dang.pdf"

def chunking_LSD_test(pdf_path):
    '''
        Hàm được sử dụng để chunk tài liệu lịch sử đảng theo từng chương, tiêu đề mục, mục con.
        Các tham số:
            - pdf_path: đường dẫn đến file pdf của môn Lịch sử đảng
            - chunk_list: Chứa các chunk đã được xử lý
        
        Hàm sẽ trả về danh sách chứa các chunk của môn LSD
    '''

    chunk_list = []
    is_content = False

    curr_chapter_line = ""
    curr_section_line = ""
    curr_sub_section_line = ""
    curr_content_line = ""

    #Duyệt qua từng trang trong file pdf, bắt đầu từ trang 2 (bỏ qua trang bìa)
    for page_layout in extract_pages(pdf_path, page_numbers=range(2, total_pages)):
        
        #Duyệt qua từng phần tử trong trang
        for element in page_layout:
            if isinstance(element, LTTextContainer):

                # Duyet qua từng dòng chữ trong phần tử
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        
                        #Lấy ra từng kí tự bên trong dòng chữ
                        for obj in text_line:
                            if isinstance(obj, LTChar):
                                '''
                                Dựa vào kích thước chữ để phân loại các phần trong tài liệu
                                - Kích thước chữ = 14: Chương và tiêu đề của chương - Done
                                - Kích thước chữ = 13, vừa in đậm vừa in nghiêng: Tiêu đề phụ (1 nhỏ, 2 nhỏ, 3 nhỏ, ...) - Done
                                - Kích thước chữ = 13, chỉ in đậm không in nghiêng: Tiêu đề lớn ( 1 la mã, 2 la mã, 3 la mã, ...), Mục Tiêu ( Chú ý xử lý trường hợp này) - Done
                                - Kích thước chữ > 12.9: Các nội dung chính
                                - Kích thước chữ = 12: Số trang - Done
                                - Kích thước chữ < 11: Nội dung phụ, chú thích, ghi chú (Loại bỏ các trường hợp này) - Done
                                '''
                                #Lấy ra thông tin của kí tự
                                font = obj.fontname
                                text = obj.get_text()
                                text_size = obj.size

                                if text_size == 14:    #Lấy ra các ký tự có kích thước chữ = 14, đây là tiêu đề chương
                                    #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                    if is_content == True:
                                        chunk = split_chunk_langchain(curr_chapter_line, curr_section_line, curr_sub_section_line, curr_content_line)
                                        chunk_list.append(chunk)
                                        curr_chapter_line = ""
                                        curr_section_line = ""
                                        curr_sub_section_line = ""
                                        curr_content_line = ""
                                        is_content = False

                                    # curr_chapter_line =""
                                    curr_chapter_line += text

                                elif text_size > 12.9 and text_size < 14:   #Lấy ra các ký tự có kích thước chữ > 12.9 (Đây là nội dung chính)

                                    #Tiêu đề la mã
                                    if ("Bold" in font) and ("Italic" not in font):
                                        #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                        if is_content == True and text != " " and text != "c" and text != ",":
                                            if "KẾT LUẬN" in curr_section_line:
                                                curr_chapter_line = "KẾT LUẬN"
                                                curr_section_line = ""
                                                curr_sub_section_line = ""
                                            chunk = split_chunk_langchain(curr_chapter_line, curr_section_line, curr_sub_section_line, curr_content_line)
                                            chunk_list.append(chunk)
                                            curr_section_line = ""
                                            curr_sub_section_line = ""
                                            curr_content_line = ""

                                            is_content = False

                                        # curr_section_line = ""
                                        curr_section_line += text
                                    
                                    #Tiêu đề phụ
                                    elif ("Italic" in font) and ("Bold" in font):
                                        #Nếu có nội dung chương trước đó thì đẩy 1 chunk
                                        if is_content == True and text != " ":
                                            chunk = split_chunk_langchain(curr_chapter_line, curr_section_line, curr_sub_section_line, curr_content_line)
                                            chunk_list.append(chunk)
                                            curr_sub_section_line = ""
                                            curr_content_line = ""
                                            is_content = False

                                        # curr_sub_section_line =""
                                        curr_sub_section_line += text
                                    
                                    #Trường hợp là văn bản bình thường
                                    else:
                                        is_content = True
                                        curr_content_line += text
    return chunk_list

In [ ]:
LSD_chunk_list = chunking_LSD_test(pdf_path)

In [41]:
LSD_chunk_list

[{'id': 'LSD-Chương nhập môn-0-0',
  'values': [],
  'metadata': {'subject': 'Lịch sử ĐCSVN',
   'chapter': 'Chương nhập môn',
   'chapter_title': 'ĐỐI TƯỢNG, CHỨC NĂNG, NHIỆM VỤ,NỘI DUNG VÀ PHƯƠNG PHÁP NGHIÊN CỨU, HỌC TẬPLỊCH SỬ ĐẢNG CỘNG SẢN VIỆT NAM',
   'section': None,
   'section_title': None,
   'subsection': None,
   'subsection_title': None,
   'content': 'Đảng Cộng sản Việt Nam do Chủ tịch Hồ Chí Minh sáng lập (3-2-1930). Từ thờiđiểm lịch sử đó, lịch sử của Đảng hòa quyện cùng lịch sử của dân tộc Việt Nam. Đảng đãlãnh đạo và đưa sự nghiệp cách mạng của giai cấp công nhân và dân tộc Việt Nam đi từthắng lợi này đến thắng lợi khác, “có được cơ đồ là vị thế như ngày nay”. “Đảng Cộngsản Việt Nam là đội tiền phong của giai cấp công nhân, đồng thời là đội tiền phong củanhân dân lao động và của dân tộc Việt Nam, đại biểu trung thành lợi ích của giai cấp côngnhân, nhân dân lao động và của dân tộc. Đảng lấy chủ nghĩa Mác-Lênin và tư tưởng HồChí Minh làm nền tảng tư tưởng, kim chỉ nam c

In [36]:
import json
import os

def save_chunks_to_json(chunks, output_path):
    """
    Lưu danh sách các chunk (list of dict) ra file JSON.
    """
    os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Tạo folder nếu chưa có

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    print(f"✅ Đã lưu {len(chunks)} chunks vào: {output_path}")

In [ ]:
output_path = r"../data/LichSuDang/Lich_Su_Dang.json"
save_chunks_to_json(LSD_chunk_list, output_path)

✅ Đã lưu 48 chunks vào: ../data/LichSuDang/Lich_Su_Dang.json


Các fontsize có trong giáo trình:
    - {5.799999999999997,
        7.5,
        7.500000000000028,
        9.999999999999986,
        10.0,
        10.000000000000007,
        11.0,
        11.000000000000007,
        12.0,
        12.999999999999943,
        12.999999999999972,
        12.999999999999993,
        13.0,
        13.000000000000007,
        14.0,
        14.000000000000007,
        14.000000000000014,
        16.0}